In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional
from keras.utils import plot_model

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

WINDOW_SIZE = 25

In [36]:
X_attack = np.load("data/lanim_attack.npy")
X_normal = np.load("data/lanim_normal.npy")

X_attack = X_attack[0:330]

print(len(X_attack))
print(len(X_normal))

y_attack = np.zeros((len(X_attack),1))
y_normal = np.ones((len(X_normal),1))

X = np.concatenate((X_attack, X_normal))
y = np.concatenate((y_attack, y_normal))

print(len(X))
print(len(y))

scalar = StandardScaler(copy=True, with_mean=True, with_std=True)
scalar.fit(X)
X = scalar.transform(X)

330
330
660
660


In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1)


In [20]:
classifiers = [
                ("KNN", KNeighborsClassifier(n_neighbors=5)),
                ("XGBoost", XGBClassifier()),
                ("Random Forest", RandomForestClassifier()),
              ]

for name, clf in classifiers:
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    print(classification_report(y_test, predictions))

c:\Users\Matheus\Desktop\REDES II - 3\.env\lib\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\Matheus\Desktop\REDES II - 3\.env\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

         0.0       0.80      1.00      0.89        36
         1.0       1.00      0.65      0.79        26

    accuracy                           0.85        62
   macro avg       0.90      0.83      0.84        62
weighted avg       0.88      0.85      0.85        62

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        36
         1.0       1.00      1.00      1.00        26

    accuracy                           1.00        62
   macro avg       1.00      1.00      1.00        62
weighted avg       1.00      1.00      1.00        62

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        36
         1.0       1.00      1.00      1.00        26

    accuracy                           1.00        62
   macro avg       1.00      1.00      1.00        62
weighted avg       1.00      1.00      1.00        62



In [38]:
X_train = X_train.reshape((len(X_train),25,26))

def create_baseline():
    model = Sequential()
    
    model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
    model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
    model.add(Dense(1, activation = 'sigmoid', kernel_regularizer='l2'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

model = create_baseline()

history = model.fit(X_train, y_train, epochs=20, validation_split=0.1, verbose = 1)


Epoch 1/20
17/17 [==============================] - 5s 57ms/step - loss: 2.3584 - accuracy: 0.9457 - val_loss: 1.8444 - val_accuracy: 1.0000
Epoch 2/20
17/17 [==============================] - 0s 10ms/step - loss: 1.5758 - accuracy: 1.0000 - val_loss: 1.3149 - val_accuracy: 1.0000
Epoch 3/20
17/17 [==============================] - 0s 10ms/step - loss: 1.1293 - accuracy: 1.0000 - val_loss: 0.9349 - val_accuracy: 1.0000
Epoch 4/20
17/17 [==============================] - 0s 10ms/step - loss: 0.7997 - accuracy: 1.0000 - val_loss: 0.6596 - val_accuracy: 1.0000
Epoch 5/20
17/17 [==============================] - 0s 11ms/step - loss: 0.5633 - accuracy: 1.0000 - val_loss: 0.4638 - val_accuracy: 1.0000
Epoch 6/20
17/17 [==============================] - 0s 10ms/step - loss: 0.3963 - accuracy: 1.0000 - val_loss: 0.3268 - val_accuracy: 1.0000
Epoch 7/20
17/17 [==============================] - 0s 10ms/step - loss: 0.2800 - accuracy: 1.0000 - val_loss: 0.2320 - val_accuracy: 1.0000
Epoch 8/20
17

In [39]:
predictions = model.predict(X_test.reshape((len(X_test),25,26)))

predictions = predictions.flatten().round()

print(classification_report(y_test, predictions))

3/3 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        33
         1.0       1.00      1.00      1.00        33

    accuracy                           1.00        66
   macro avg       1.00      1.00      1.00        66
weighted avg       1.00      1.00      1.00        66



In [40]:
model.save("models/LANIM-330.h5")

c:\Users\Matheus\Desktop\REDES II - 3\.env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
